In [ ]:
%run ../utilities/web_utilities.ipynb
%run ../utilities/database_utilities.ipynb
%run ../utilities/encoder_utilities.ipynb
%run ../utilities/pd_utilities.ipynb

In [ ]:
write_mode = 'replace'
target_table = 'whl_player_season_stats'

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../../')

import settings as st

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
db_host = st.database['local']['host']
db_port = st.database['local']['port']
db_user = st.database['local']['user']
db_pass = st.database['local']['pass']
db_sys = st.database['local']['system']
db_db = st.database['local']['db']

In [ ]:
key = "41b145a848f4bd67"

In [ ]:
sql = "SELECT DISTINCT player_id FROM whl_team_roster_by_season WHERE player_id IS NOT NULL ORDER BY player_id ASC"
res = query_database_to_list(db_sys, db_user, db_pass, db_host, db_port, db_db, sql)

In [ ]:
master_df = pd.DataFrame()

In [ ]:
i = 1
for r in res:
    player_id = str(r['player_id']).replace(".0", "")
    
    print(i, player_id)
    
    url = "http://lscluster.hockeytech.com/feed/?feed=modulekit&view=player&key={0}&fmt=json&client_code=whl&lang=en&player_id={1}&category=seasonstats".format(key, player_id)
    
    json_data = return_json(url)
    
    player_df = pd.DataFrame()
    
    try:
        playoff_df = pd.DataFrame(json_data['SiteKit']['Player']['playoff'])
        playoff_df = playoff_df[playoff_df.shortname != "total"]
        playoff_df['season_type'] = 'Playoff'
        
        player_df = player_df.append(playoff_df)
    except:
        pass
    
    try:
        regular_df = pd.DataFrame(json_data['SiteKit']['Player']['regular'])
        regular_df = regular_df[regular_df.shortname != "total"]
        regular_df['season_type'] = 'Regular'

        player_df = player_df.append(regular_df)
    except:
        pass
    
    master_df = master_df.append(player_df)
    
    i+=1

In [ ]:
master_df = master_df.reset_index(drop=True)

In [ ]:
if write_mode == 'append':
    index = get_table_new_id(db_sys, db_user, db_pass, db_host, db_port, db_db, target_table, 'index')
    master_df.insert(0, 'index', master_df.index + index)
else:
    master_df.insert(0, 'index', master_df.index)

In [ ]:
master_df

In [ ]:
write_df_to_database(master_df, target_table, db_sys, db_user, db_pass, db_host, db_port, db_db, write_mode, False)